<a href="https://colab.research.google.com/github/jeanbouteiller-ds/tennis_prediction/blob/main/functions_fixed_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import subprocess
import sys

def install_package(package_name):
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package_name])
        print(f"Successfully installed {package_name}")
    except subprocess.CalledProcessError:
        print(f"Failed to install {package_name}") # Install the PyGitHub library

install_package("requests")
install_package("PyGitHub")

In [4]:
import requests
import random
from bs4 import BeautifulSoup

from datetime import datetime, timedelta

import os
from github import Github
from io import StringIO


#Set Up Github Environment

In [5]:
# Replace these with your GitHub credentials
username = "jeanbouteiller-ds"

github_url = "https://raw.githubusercontent.com/jeanbouteiller-ds/tennis_prediction/main/password.txt"
response = requests.get(github_url)

password= response.text.replace('\n','')
repository_name = "tennis_prediction"
branch_name = "main"  # Replace with your branch name


# Initialize the GitHub client
g = Github(username, password)

# Access the desired repository
repo = g.get_user().get_repo(repository_name)


def add_file_github(file_name,file_content):
  try:
      file = repo.get_contents(file_name, ref=branch_name)
      sha = file.sha
  except Exception as e:
      sha = None

  if sha:
      repo.update_file(file_name, "Update file", file_content, sha, branch_name)
      # print(f"Updated {file_name} in {repository_name}")
  else:
      repo.create_file(file_name, "Create file", file_content, branch_name)
      # print(f"Created {file_name} in {repository_name}")

def get_file_content(file_name):
  file_content = repo.get_contents(file_name, ref=branch_name)
  file_content = file_content.decoded_content.decode("utf-8")
  if file_name.split('.')[1]=='txt':
    file_content=eval(file_content)
    return(file_content)
  if file_name.split('.')[1]=='csv':
    file_content=pd.read_csv(StringIO(file_content))
    return(file_content)

# Get the list of all players and their associated urls

In [6]:
#get the list of all first days of week between 2 dates
#needed since the url on the atp website is updated every monday
def get_list_ranking_dates(start_date, end_date):
    mondays = []
    current_date = start_date
    one_day = timedelta(days=1)
    while current_date <= end_date:
        if current_date.weekday() == 0:  # Monday has a weekday index of 0
            mondays.append(current_date.strftime('%Y-%m-%d'))
        current_date += one_day
    return mondays

#get the list of all urls in which we have the rankings
def find_list_weekly_ranking_url(start_date, end_date):
  list_ranking_urls=[]
  #date should be in format YYYY-MM-DD and should be a string
  list_dates=get_list_ranking_dates(start_date, end_date)

  for date_ranking in list_dates:
    list_ranking_urls.append('https://www.atptour.com/en/rankings/singles?rankRange=1-200&rankDate='+date_ranking)
  return(list_ranking_urls)



In [7]:
#for each url, we extract the spans with the url associated with all players
def span_from_url(weekly_ranking_url,span_list):
  headers = {
      'User-Agent': 'Your User Agent String Here'+str(random.random())
  }

  # Send an HTTP GET request with headers
  response = requests.get(weekly_ranking_url, headers=headers)

  # Check if the request was successful (status code 200)
  if response.status_code == 200:
      # Get the HTML content from the response
      html_code = response.text
  else:
      html_code=''

  soup = BeautifulSoup(html_code, 'html.parser')
  selected_span={}

  for span_name in span_list:
    selected_span[span_name] = soup.find_all('span', class_=span_name)

  return (selected_span)

#from the above spans, we extract all urls
def players_from_url(weekly_ranking_url):

  selected_span=span_from_url(weekly_ranking_url,['player-cell-wrapper'])['player-cell-wrapper']

  list_urls=[]
  list_players=[]

  for span_element in selected_span:
    a_element = span_element.find('a')

      # Extract the href attribute
    if a_element:
      # print(a_element)
      href = 'https://www.atptour.com'+a_element.get('href')
      player_name=href.split('players')[1].split('/')[1]
      list_players.append(player_name)
      list_urls.append(href)
    else:
        print("No <a> element found within the <span>.")
  return(list_players,list_urls)

#
def update_names_and_urls(weekly_ranking_url,list_all_names,list_all_urls):
  list_players_name_and_url=players_from_url(weekly_ranking_url)
  set_names_week = set(list_players_name_and_url[0])
  set_urls_week = set(list_players_name_and_url[1])
  set_names_all = set(list_all_names)
  set_urls_all = set(list_all_urls)

  names_to_add=list(set_names_week-set_names_all)
  urls_to_add=list(set_urls_week-set_urls_all)

  return(list_all_names+names_to_add,
         list_all_urls+urls_to_add)

In [10]:
def create_initial_list(start_date,end_date):
  list_all_names=[]
  list_all_urls=[]

  # Define your start and end dates
    # Change this to your desired end date

  list_year=[]
  for weekly_url_ranking in find_list_weekly_ranking_url(start_date, end_date):
    if weekly_url_ranking.split('rankDate=')[1].split('-')[0] not in list_year:
      list_year.append(weekly_url_ranking.split('rankDate=')[1].split('-')[0])
      # print(list_year)
    names_and_url_week=update_names_and_urls(weekly_url_ranking,list_all_names,list_all_urls)
    list_all_names=names_and_url_week[0]
    list_all_urls=names_and_url_week[1]

  list_all_urls_final=[0 for k in range(len(list_all_urls))]
  for k in range(len(list_all_names)):
    player_name=list_all_names[k]
    for player_url in list_all_urls:
      if player_name==player_url.split('/players/')[1].split('/')[0]:
        list_all_urls_final[k]=player_url

  return(list_all_names,list_all_urls_final)




# Get Players Data

In [14]:
def find_stats_from_span(url,span_names_list=['table-height-cm-wrapper','table-weight-lbs','table-birthday'],
               feature_names=['height','weight (lbs)','Birthdate']):
  dict_data={}
  data_spans=span_from_url(url,span_names_list)
  for k in range (len(feature_names)):
    key=feature_names[k]
    values=list(data_spans.values())[k][0].get_text()
    dict_data[key]=values.replace(' ','').replace('\r\n','')
  return dict_data

In [15]:
def find_stats_from_div(url,div_dict={'Turned Pro':['table-big-label','table-big-value'],
                                      'Plays':['table-label','table-value']}):
  #div_dict should be a dict where the key is the label that we want to find in the html code
  # and the values are the div that we want to search (for label and values)
  dict_data={}
  headers = {
      'User-Agent': 'Your User Agent String Here'+str(random.random())
  }

  # Send an HTTP GET request with headers
  response = requests.get(url, headers=headers)

  # Check if the request was successful (status code 200)
  if response.status_code == 200:
      # Get the HTML content from the response
      html_code = response.text
  else:
      html_code=''

  soup = BeautifulSoup(html_code, 'html.parser')

  for feature_name in div_dict.keys():

    div_name=div_dict[feature_name][0]
    div_value=div_dict[feature_name][1]

    for div in soup.find_all('div', class_=div_name):
      # print(div)
      if feature_name in div.get_text():

        dict_data[feature_name]=(div.find_next_sibling('div', class_=div_value).get_text()).replace(' ','').replace('\r\n','')
        # if
        # print(div.find_next_sibling('div', class_='table-big-value').get_text())

  return (dict_data)


def find_all_stats(url):
  return({**find_stats_from_div(url),**find_stats_from_span(url)})

In [24]:
# import pandas as pd
# def create_new_fixed_table(file_name,list_player_names,list_urls):
#   df_player_fixed=pd.DataFrame()
#   nb_error=0
#   for player_k in range(len(list_player_names)):
#     url=list_urls[player_k]
#     player_name=list_player_names[player_k]
#     try:
#       df_player_fixed[player_name]=find_all_stats(url)
#     except Exception as e:
#       nb_error+=1

#   csv_player_fixed=df_player_fixed.T.to_csv(index=True)

#   add_file_github(file_name,csv_player_fixed)

In [ ]:
import pandas as pd
import concurrent.futures

def fetch_player_stats(url, player_name):
    try:
        stats = find_all_stats(url)
        return (player_name, stats)
    except Exception as e:
        return (player_name, None)

def create_new_fixed_table(file_name, list_player_names, list_urls):
    player_data = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(fetch_player_stats, url, player_name) for url, player_name in zip(list_urls, list_player_names)]

        for future in concurrent.futures.as_completed(futures):
            player_name, stats = future.result()
            if stats is not None:
                player_data.append((player_name, stats))

    df_player_fixed = pd.DataFrame(player_data, columns=['Player Name', 'Stats'])

    # You can save the DataFrame to a file if needed.
    df_player_fixed.to_csv(file_name, index=False)

    return df_player_fixed

# Usage
# df = create_new_fixed_table("output.csv", list_player_names, list_urls)
